In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import List, Dict, Optional

c:\walker\code\langchain-basic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from schedule_toolkits import ScheduleToolkit # __init__ 파일이 있어서 from schedule_toolkits.toolkit이라고 쓸 필요가 없는 것 
from schedule_toolkits import AddTodoTool
from schedule_toolkits import ViewTodoTool

In [4]:
# 1. tool setting
tools = ScheduleToolkit().get_tools()
tools

[AddTodoTool(), ViewTodoTool()]

In [5]:
# 2. llm setting
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000026AF3A63BD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026AF50C8A10>, root_client=<openai.OpenAI object at 0x0000026AF3C07ED0>, root_async_client=<openai.AsyncOpenAI object at 0x0000026AF50C86D0>, model_name='gpt-4.1-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [6]:
# 3. prompt setting
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '너는 일정을 잘 관리하는 ai 비서야. 주어지는 tool을 활용해서 사용자의 요청을 처리해줘'),
        MessagesPlaceholder(variable_name='history'),
        ('human', '{question}'),
        ("placeholder", "{agent_scratchpad}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['history', 'question'], optional_variables=['agent_scratchpad'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemM

In [7]:
# 4. agent setting
agent = create_openai_tools_agent(
    model,
    tools,
    prompt
)

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [8]:
# 5. memory setting
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

stores : Dict[str, InMemoryChatMessageHistory] = {}

def get_store(session_id : str):
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]

agent_history = RunnableWithMessageHistory(
    executor,
    lambda session_id: get_store(session_id),
    input_messages_key="question",
    history_messages_key="history"
)

In [ ]:
# 6. schedule register
cfg = {"configurable" : {"session_id" : "user-123"}}

agent_history.invoke({
        "question" : "오늘 오후 5시에 미팅 일정 등록해줘"
    }, config = cfg
)



> Entering new AgentExecutor chain...

Invoking: `add_todo` with `{'item': '돌마리 운동회 12시 30분 파워댄스'}`


Added 돌마리 운동회 12시 30분 파워댄스 to schedule오늘 돌마리 운동회 12시 30분에 파워댄스 일정을 등록해드렸습니다. 다른 일정도 도와드릴까요?

> Finished chain.


{'question': '오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘',
 'history': [],
 'output': '오늘 돌마리 운동회 12시 30분에 파워댄스 일정을 등록해드렸습니다. 다른 일정도 도와드릴까요?'}

In [10]:
# 6. schedule viewer
cfg = {"configurable" : {"session_id" : "user-123"}}

agent_history.invoke({
    "question" : "등록된 일정들 알려줘"
    }, config = cfg
)



> Entering new AgentExecutor chain...

Invoking: `view_todos` with `{}`


test item : 돌마리 운동회 12시 30분 파워댄스
False
Todo Schedule(Not Completed): - 돌마리 운동회 12시 30분 파워댄스등록된 일정은 "돌마리 운동회 12시 30분 파워댄스"입니다. 다른 일정도 확인하거나 추가할 것이 있나요?

> Finished chain.


{'question': '등록된 일정들 알려줘',
 'history': [HumanMessage(content='오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘 돌마리 운동회 12시 30분에 파워댄스 일정을 등록해드렸습니다. 다른 일정도 도와드릴까요?', additional_kwargs={}, response_metadata={})],
 'output': '등록된 일정은 "돌마리 운동회 12시 30분 파워댄스"입니다. 다른 일정도 확인하거나 추가할 것이 있나요?'}